# Сравнение open-source солверов и их применение в ритейле

Оптимизация или как ее еще называют математическое программирование - это, пожалуй, один из ключевых разделов теоретической и прикладной математики, который находит свое место практически во всех аспектах нашей жизни. Действительно, каждый человек, не задумываясь об этом, практически каждый день решает оптимизационные задачи. За примерами далеко ходить не надо - все мы часто ходим в магазин или совершаем покупки в онлайн, соответственно, мы как правило хотим минимизировать наши расходы для приобретения необходимых товаров или же при ограниченном бюджете приобрести как можно больше товаров согласно приоритетам. Или же мы хотим распределить свое время так, что бы это было максимально эффективно с какой-то точки зрения. Также оптимизация используется во всех современных алгоритмах ML для обучения, подбора параметром алгоритмов. Как правило ML модели или подобного рода алгоритмы позволяют оценивать эффекты от какого либо воздействия, поэтому может возникнуть задача оптмального управления, где методы оптимизации будут чрезвычайно полезны или даже необходимы для принятия решений. 

Формулировка оптимизационной задачи может быть нетривиальным процессом, часто требующие определенной экспертизы, в том числе и в предметной области - что хотим максимизировать/минимизировать, какие условия надо соблюсти и т.п. также надо понимать, а какие есть инструменты для решения задачи.

Тему связанную с оптимизацией объять одной статьей невозможно, так как существует очень много классов задач и, соответственно, методов их решения.
Здесь постараемся рассмотреть некоторые постановки задач оптимизации в ритейле и разберем поподробнее на практически важной задаче Ценообразования возможные постановки и сравним некоторые подходы к решению, основанные на открытых библиотеках в python


Прежде всего рассмотрим постановку задачи в общем виде:

\
$x$ - вектор размерностью $n$

$f(x) \to \min(\max)$ - целевая функция

$g_i(x) \leqslant 0, \ i=1..m$ - ограничения вида неравенства

$h_i(x) = 0, \ j=1..k$ - ограничения вида равенства

$x \in X$ - допустимое множество значений переменных ($ \mathbb{R}, \mathbb{Z}$ и т.п.)

\
Исходя из практики можно разложить данную постановку на несколько классов в зависимости от вида целевой функции, ограничений и $X$

* __Безусловная оптимизация__ $g_i(x), h_j(x)$ - отсутствуют, $X = \mathbb{R}^n$

* __LP__ (linear programming) - линейное программирование. $f(x), g_i(x), h_j(x)$ - линейные функции, $X = \mathbb{R}_+^n$. 
$\mathbb{R}_+$

* __MILP__ (mixed integer linear programming) - смешанное целочисленное линейное программирование, это задача LP в которой часть переменных являются целочисленными

* __NLP__ (nonlinear programming) - нелинейное програмирование, возникает когда хотя бы одна из функций $f(x),\ g_i(x),\ h_j(x)$ нелинейна

* __MINLP__ (mixed integer nonlinear programming) - смешанное целочисленное нелинейное программирование, по аналогии с __MILP__ в случае наличия целочисленных переменных


__NLP__ в свою очередь можно подробить еще на кучу разных классов в зависимости от вида нелинейности и выпуклости


### Примеры из области ритейла.

#### Оптимальное распределение маркетингового бюджета
Реализовать выделенный бюджет на маркетинговые активности максимально эффективно. Есть несколько каналов для рекламных акций, выделенный бюджет, цель - максимально выгодно инвестировать бюджет, чтобы суммарный доход со всем коммуникаций был максимален. Также необходимо учесть бизнес ограничения на нагрузку каждого канала + частоту взаимодействия.


#### Планирование ассортимента

Подобрать ассортимент


#### Закупка товаров
Задача - распределить бюджет выделенный на закупки для поддержания товарооборота, достаточного уровня сервиса, и при этом достигать определенных финансовых показателей под выделенный бюджет на закупки

#### Ценообразование [ссылка на статью].
Поиск оптимальной конфигурации цен с учетом ценового позиционирования, допустимых ценовых диапазонов для каждого товара и набора бизнес-ограничений. Цены должны максимизировать суммарный доход, а прибыль быть не ниже на заданного уровня.


## Обзор пакетов python
В указанных выше задачах возникает условная оптимизация, здесь предлагается рассмотреть полезные open-source пакеты и солверы для решения задач условной оптимизации в зависимости от ее типа, которые часто можно встретить в практических задачах


### Открытые библиотеки, предоставляющие интерфейс для решения оптимизационных задач

__Scipy__ - библиотека, которая содержит большой набор функций для научных вычислений, в том числе имеет инструменты для решения оптимизационных задач, находящиеся в модуле scipy.optimize. В модуле находятся методы для решения задач линейного программирования, нелинейного программирования(как условная так и безусловная). 
В документации [scipy.optimize.minimize](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html) можно найти, что условную оптимизацию с нелинейнойстью поддерживают __cobyla__, __slsqp__, __trust-constr__, также там можно найти краткое описание методов и ссылки на статьи, [статья на хабре](https://habr.com/ru/company/ods/blog/448054/)

[__Pyomo__](http://www.pyomo.org/) - пакет, который содержит ряд инструментов для формулирования, решения и анализа оптимизационных моделей. Главная особенность - это удобный интерфейс для структурированного формулирования оптимизационной задачи и поддержка большого количества солверов, в том числе коммерческих. По сути pyomo занимается “перевариванием” сформулированной модели в формат, понятный для запускаемого солвера, потом забирает его и выплевывает в интерфейс python. Является частью проекта [COIN-OR](https://www.coin-or.org/), который содержит также ряд солверов. Среди них можно выделить [Ipopt](https://github.com/coin-or/Ipopt), [Cbc](https://github.com/coin-or/Cbc). Ipopt позволяет находить локальные оптимумы в задаче NLP с помощью прямо-двойственного метода внутренней точки, подробнее в оригальной [статье](http://www.optimization-online.org/DB_HTML/2004/03/836.html). Cbc - в нем реализован метод решения задачи MILP, основанный на алгоритме сочетающий в себе метод ветвей и границ и секущих плоскостей [wiki](https://en.wikipedia.org/wiki/Branch_and_cut). Также для LP имеется поддержка пакета [glpk](https://en.wikipedia.org/wiki/GNU_Linear_Programming_Kit).

[__Cvxpy__](https://www.cvxpy.org/index.html) - данный пакет специально заточен для решения задач выпуклой оптимизации (convex optimization). После того как задача сформулирована, перед решением проверяется выпуклость и аффинность целевой функции и ограничений с помощью правил [DCP](https://www.cvxpy.org/tutorial/dcp/index.html#:~:text=Disciplined%20convex%20programming%20(DCP)%20is,they%20are%20applied%20by%20CVXPY) (disciplined convex programming), после проверки задача преобразуется в стандартную форму и передается квадратичному или коническому солверу. Полный список солверов и типов задач, с помощью которых можно решить их можно найти [здесь](https://www.cvxpy.org/tutorial/advanced/index.html)

Итого можно сформировать таблицу с солверами, библиотеками и видами задач, которые позволяют решать солверы

| Солвер(метод) | Пакеты в python | NLP | LP | MILP | MINLP |
|:-------------:|:---------------:|:---:|:--:|:----:|:-----:|
|     cobyla    |      scipy      |  y  |  n |   n  |    n  |
|     slsqp     |      scipy      |  y  |  n |   n  |    n  |
|  trust-constr |      scipy      |  y  |  n |   n  |    n  |
|     ipopt     |      pyomo      |  y  |  y |   n  |    n  |
|      glpk     |   pyomo, cvxpy  |  n  |  y |   y  |    n  |
|      cbc      |   pyomo, cvxpy  |  n  |  y |   y  |    n  |

### Модельная задача оптимизации в ценообразовании

Как уже было сказано выше, одной из основных целей ценообразования может быть формирование таких цен, которые:

* Соответствуют некоторому набору правил, бизнес-логике и т.п.

* Позволяют достигать определенных целей, например, повышение выручки, трафика, без просадки в других показателях.

Первый пункт по сути задает нам диапазон в котором производится поиск новой цены, возможные соотношения между ценами на товары, соотношения цен с ценами конкурентов, условия на (не)изменение цены.

Второй задает нам целевой функционал для задачи оптимизации и набор ограничений, которые контролируют остальные показатели.

Чтобы начать решать задачу требуется знать отклик измененеия спроса на изменение цены, здесь мы не будем отдельно останавливаться на том как оценивать этот отклик, будем считать что он задан в следующем виде:
$$Q = Q_{0} \exp\bigg(E \cdot \bigg(\frac{P}{P_0} - 1\bigg)\bigg)$$
, где $Q$ - спрос по новой цене $P$, $Q_0$ - спрос по текущей цене $P_{0}$ , $E$ - коэффициент эластичности. 
Для удобства введем переменую $x = \frac{P}{P_0}$, тогда функцию спроса можно переписать в следующем виде.
$$Q = Q_0 \exp(E \cdot (x-1))$$

